In [2]:
# pip install xgboost scikit-learn pandas seaborn tensorflow

import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [3]:
titanic = sns.load_dataset('titanic').dropna(
    subset=['age', 'sex', 'class', 'survived'])
# features que usamos para tentar prever algo
X = titanic[['age', "sex", "class"]]
y = titanic['survived']  # o que queremos prever

In [4]:
# Pre processing
X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [5]:
# ======================
# 🔹 Model 1: XGBoost
# ======================
model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict(X_test)
print("XGBoost accuracy:", accuracy_score(y_test, y_pred))

/home/diogo/miniconda3/envs/vertexai_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:44:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost accuracy: 0.7902097902097902


In [6]:
# ======================
# 🔹 Model 2: RNN
# ======================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# RNN the data must be in 3D shape (samples, timesteps, features)
X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_rnn = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))


In [7]:

model_rnn = Sequential([
    SimpleRNN(16, activation="tanh", input_shape=(1, X_train.shape[1])),
    Dense(1, activation="sigmoid")
])

model_rnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_rnn.fit(X_train_rnn, y_train, epochs=50, batch_size=8, verbose=0)

# Avaliar RNN
loss, acc_rnn = model_rnn.evaluate(X_test_rnn, y_test, verbose=0)
print("Acurácia RNN:", acc_rnn)

Acurácia RNN: 0.7412587404251099
